### Desabilitando os warnings

In [1]:
import warnings
warnings.filterwarnings('ignore')

### Importando bibliotecas

In [2]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re
import plotly.express as px

### Importando conjuntos de dados

In [3]:
planilhas = pd.read_excel(
    r"..\data\Global Superstore.xls", 
    sheet_name=None)

### Tratando conjunto de dados Orders

##### Analisando conjunto de dados

In [4]:
df_orders = planilhas["Orders"] 
df_orders.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


In [5]:
df_orders["Row ID"] = df_orders["Row ID"].apply(pd.to_numeric, downcast="integer")
df_orders.set_index("Row ID", inplace=True)

In [6]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51290 entries, 32298 to 6147
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        51290 non-null  object        
 1   Order Date      51290 non-null  datetime64[ns]
 2   Ship Date       51290 non-null  datetime64[ns]
 3   Ship Mode       51290 non-null  object        
 4   Customer ID     51290 non-null  object        
 5   Customer Name   51290 non-null  object        
 6   Segment         51290 non-null  object        
 7   City            51290 non-null  object        
 8   State           51290 non-null  object        
 9   Country         51290 non-null  object        
 10  Postal Code     9994 non-null   float64       
 11  Market          51290 non-null  object        
 12  Region          51290 non-null  object        
 13  Product ID      51290 non-null  object        
 14  Category        51290 non-null  object        
 15  Sub-

In [7]:
df_orders.select_dtypes(exclude="number").describe()

,Order Date,Ship Date
count,51290,51290
mean,2013-05-11 21:26:49.155781120,2013-05-15 20:42:42.745174528
min,2011-01-01 00:00:00,2011-01-03 00:00:00
25%,2012-06-19 00:00:00,2012-06-23 00:00:00
50%,2013-07-08 00:00:00,2013-07-12 00:00:00
75%,2014-05-22 00:00:00,2014-05-26 00:00:00
max,2014-12-31 00:00:00,2015-01-07 00:00:00


In [8]:
df_orders.select_dtypes(include="number").describe()

,Postal Code,Sales,Quantity,Discount,Profit,Shipping Cost
count,9994.000000,51290.000000,51290.000000,51290.000000,51290.000000,51290.000000
mean,55190.379428,246.490581,3.476545,0.142908,28.610982,26.375818
std,32063.693350,487.565361,2.278766,0.212280,174.340972,57.296810
min,1040.000000,0.444000,1.000000,0.000000,-6599.978000,0.002000
25%,23223.000000,30.758625,2.000000,0.000000,0.000000,2.610000
50%,56430.500000,85.053000,3.000000,0.000000,9.240000,7.790000
75%,90008.000000,251.053200,5.000000,0.200000,36.810000,24.450000
max,99301.000000,22638.480000,14.000000,0.850000,8399.976000,933.570000


##### Removendo colunas irrelevantes

In [9]:
df_orders = df_orders.drop("Postal Code", axis=1)

##### Tratando valores ausentes

In [10]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51290 entries, 32298 to 6147
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        51290 non-null  object        
 1   Order Date      51290 non-null  datetime64[ns]
 2   Ship Date       51290 non-null  datetime64[ns]
 3   Ship Mode       51290 non-null  object        
 4   Customer ID     51290 non-null  object        
 5   Customer Name   51290 non-null  object        
 6   Segment         51290 non-null  object        
 7   City            51290 non-null  object        
 8   State           51290 non-null  object        
 9   Country         51290 non-null  object        
 10  Market          51290 non-null  object        
 11  Region          51290 non-null  object        
 12  Product ID      51290 non-null  object        
 13  Category        51290 non-null  object        
 14  Sub-Category    51290 non-null  object        
 15  Prod

OBS.: não há valores ausentes nas colunas selecionadas

#### Tratando tipos de dados e possíveis inconsistências de registro

In [11]:
df_orders.columns

Index(['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Segment', 'City', 'State', 'Country', 'Market',
       'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost',
       'Order Priority'],
      dtype='object')

##### Colunas com ID`s

In [12]:
print(f"Qtd. Valores Únicos na coluna Order ID : {len(df_orders["Order ID"].unique())}")
print(f"Qtd. Valores Únicos na coluna Customer ID : {len(df_orders["Customer ID"].unique())}")
print(f"Qtd. Valores Únicos na coluna Product ID : {len(df_orders["Product ID"].unique())}")

Qtd. Valores Únicos na coluna Order ID : 25035
Qtd. Valores Únicos na coluna Customer ID : 1590
Qtd. Valores Únicos na coluna Product ID : 10292


In [13]:
# Analisando valores da coluna Order ID
for id in df_orders["Order ID"].unique():
    print(id)

CA-2012-124891
IN-2013-77878
IN-2013-71249
ES-2013-1579342
SG-2013-4320
IN-2013-42360
IN-2011-81826
IN-2012-86369
CA-2014-135909
CA-2012-116638
CA-2011-102988
ID-2012-28402
SA-2011-1830
MX-2012-130015
IN-2013-73951
ES-2014-5099955
CA-2014-143567
ES-2014-1651774
IN-2014-11763
TZ-2014-8190
PL-2012-7820
CA-2011-154627
IN-2011-44803
ES-2013-2860574
US-2014-133193
MX-2014-165309
IN-2011-10286
ES-2011-4699764
CA-2013-159016
IN-2012-44810
US-2011-128776
ES-2012-5870268
CA-2012-139731
IN-2011-28087
CA-2011-168494
CG-2011-8610
CA-2011-160766
US-2014-168116
ES-2014-2637201
IN-2011-61302
ID-2013-63976
IN-2014-37320
IN-2014-76016
ES-2012-5877219
IT-2011-3183678
CA-2011-116904
IT-2013-3085011
IN-2014-50473
IN-2014-35983
MX-2014-126984
US-2012-163825
IR-2014-8540
US-2014-135013
MZ-2013-3690
IN-2012-66342
CA-2012-111829
IN-2012-48240
IN-2014-61792
CA-2014-129021
IN-2014-75470
CA-2012-114811
ES-2014-4673578
IT-2013-3376681
IN-2014-66615
CA-2013-143805
IT-2014-4540740
IN-2014-11231
MX-2014-154907
UP-20

In [14]:
# Analisando valores da coluna Customer ID
for id in df_orders["Customer ID"].unique():
    print(id)

RH-19495
JR-16210
CR-12730
KM-16375
RH-9495
JM-15655
TS-21340
MB-18085
JW-15220
JH-15985
GM-14695
AJ-10780
MM-7260
VF-21715
PF-19120
BP-11185
TB-21175
PJ-18835
JS-15685
RH-9555
AB-600
SA-20830
JK-15325
LB-16795
NP-18325
VD-21670
PB-19210
EB-14110
KF-16285
BP-11230
RR-19525
BS-11365
JE-15745
DP-13105
NP-18700
AH-30
DM-13015
GT-14635
PO-18865
DL-12865
JB-16000
BF-11005
VG-21805
GT-14710
ZC-21910
SC-20095
EB-13840
AP-10915
SW-20275
JH-15820
LC-16885
TG-11640
HR-14830
DG-3300
SG-20470
FH-14365
GP-14740
MW-18220
PO-18850
CS-12460
KD-16495
MS-17980
KC-16675
DB-13405
JD-15895
DK-13090
CS-11845
EM-14200
DW-3480
AS-10225
ST-20530
BW-1065
LA-16780
AB-10150
CM-12385
BE-11410
SZ-20035
RM-19375
HG-14845
DP-3105
AR-10540
TP-21415
AM-10705
AB-255
RP-19270
BK-11260
JM-15250
LW-16990
NZ-18565
SV-10815
SC-20695
NC-18625
NF-18385
PM-18940
MZ-17335
HM-4980
RW-19540
LH-17155
KM-16660
AB-10060
DB-13060
VM-21685
DM-13525
NM-18520
RO-19780
SB-20185
EM-14065
RS-19765
ON-18715
VS-21820
CM-12115
TB-21400
AB-1010

In [15]:
# Analisando valores da coluna Product ID
for id in df_orders["Product ID"].unique():
    print(id)

TEC-AC-10003033
FUR-CH-10003950
TEC-PH-10004664
TEC-PH-10004583
TEC-SHA-10000501
TEC-PH-10000030
FUR-CH-10004050
FUR-TA-10002958
OFF-BI-10003527
FUR-TA-10000198
OFF-SU-10002881
FUR-TA-10001889
TEC-CIS-10001717
FUR-CH-10002033
OFF-AP-10003500
OFF-AP-10000423
TEC-AC-10004145
OFF-AP-10004512
TEC-CO-10000865
OFF-KIT-10004058
FUR-HON-10000224
TEC-PH-10001363
FUR-CH-10000027
OFF-AP-10003590
TEC-PH-10004182
FUR-TA-10002827
FUR-TA-10004744
TEC-PH-10002885
FUR-CH-10001415
TEC-PH-10002815
TEC-MA-10000161
FUR-CH-10002024
OFF-AP-10004246
FUR-TA-10003473
TEC-APP-10000308
TEC-MA-10003979
TEC-MA-10004125
TEC-CO-10000013
TEC-PH-10000499
TEC-PH-10003856
FUR-BO-10004852
OFF-AP-10000486
OFF-BI-10001120
FUR-CH-10003365
FUR-CH-10000602
TEC-MA-10002680
FUR-CH-10000891
TEC-CAN-10003392
TEC-CO-10001449
FUR-BO-10001372
TEC-MOT-10002272
TEC-CO-10004997
TEC-CO-10001766
FUR-TA-10000226
OFF-AP-10002244
TEC-PH-10001459
FUR-BO-10001073
TEC-MA-10000045
TEC-PH-10002035
TEC-PH-10002565
OFF-AP-10002945
FUR-BO-10004999
F

In [16]:
df_orders["Order ID"] = df_orders["Order ID"].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z0-9-]', '', unidecode(x.upper())))
df_orders["Customer ID"] = df_orders["Customer ID"].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z0-9-]', '', unidecode(x.upper())))
df_orders["Product ID"] = df_orders["Product ID"].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z0-9-]', '', unidecode(x.upper())))

In [17]:
print(f"Qtd. Valores Únicos na coluna Order ID : {len(df_orders["Order ID"].unique())}")
print(f"Qtd. Valores Únicos na coluna Customer ID : {len(df_orders["Customer ID"].unique())}")
print(f"Qtd. Valores Únicos na coluna Product ID : {len(df_orders["Product ID"].unique())}")

Qtd. Valores Únicos na coluna Order ID : 25035
Qtd. Valores Únicos na coluna Customer ID : 1590
Qtd. Valores Únicos na coluna Product ID : 10292


##### Colunas com nomes

In [18]:
print(f"Qtd. Valores Únicos na coluna Customer Name : {len(df_orders["Customer Name"].unique())}")
print(f"Qtd. Valores Únicos na coluna Product Name : {len(df_orders["Product Name"].unique())}")

Qtd. Valores Únicos na coluna Customer Name : 795
Qtd. Valores Únicos na coluna Product Name : 3788


In [19]:
# Analisando valores da coluna Customer Name
for nome in df_orders["Customer Name"].unique():
    print(nome)

Rick Hansen
Justin Ritter
Craig Reiter
Katherine Murray
Jim Mitchum
Toby Swindell
Mick Brown
Jane Waco
Joseph Holt
Greg Maxwell
Anthony Jacobs
Magdelene Morse
Vicky Freymann
Peter Fuller
Ben Peterman
Thomas Boland
Patrick Jones
Jim Sink
Ritsa Hightower
Ann Blume
Sue Ann Reed
Jason Klamczynski
Laurel Beltran
Naresj Patel
Valerie Dominguez
Phillip Breyer
Eugene Barchas
Karen Ferguson
Benjamin Patterson
Rick Reed
Bill Shonely
Joel Eaton
Dave Poirier
Nora Preis
Aaron Hawkins
Darrin Martin
Grant Thornton
Patrick O'Donnell
Dan Lawera
Joy Bell-
Barry Franz
Vivek Grady
Greg Tran
Zuschuss Carroll
Sanjit Chand
Ellis Ballard
Arthur Prichep
Scott Williamson
John Huston
Lena Creighton
Trudy Glocke
Harold Ryan
Deirdre Greer
Sheri Gordon
Fred Hopkins
Guy Phonely
Mitch Webber
Patrick O'Brill
Chuck Sachs
Keith Dawkins
Michael Stewart
Kimberly Carter
Denny Blanton
Jonathan Doherty
Dave Kipp
Cari Sayre
Evan Minnotte
Dianna Wilson
Alan Schoenberger
Shui Tom
Barry Weirich
Laura Armstrong
Aimee Bixby
Christ

In [20]:
# Analisando valores da coluna Product Name
for nome in df_orders["Product Name"].unique():
    print(nome)

Plantronics CS510 - Over-the-Head monaural Wireless Headset System
Novimex Executive Leather Armchair, Black
Nokia Smart Phone, with Caller ID
Motorola Smart Phone, Cordless
Sharp Wireless Fax, High-Speed
Samsung Smart Phone, with Caller ID
Novimex Executive Leather Armchair, Adjustable
Chromcraft Conference Table, Fully Assembled
Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind
Chromcraft Bull-Nose Wood Oval Conference Tables & Bases
Martin Yale Chadless Opener Electric Letter Opener
Bevis Conference Table, Fully Assembled
Cisco Smart Phone, with Caller ID
Harbour Creations Executive Leather Armchair, Adjustable
KitchenAid Microwave, White
Breville Refrigerator, Red
Logitech diNovo Edge Keyboard
Hoover Stove, Red
Brother Fax Machine, High-Speed
KitchenAid Stove, White
Hon Computer Table, with Bottom Storage
Apple iPhone 5S
SAFCO Executive Leather Armchair, Black
KitchenAid Refrigerator, Black
Motorola Smart Phone, Full Size
Hon Computer Table, Fully Assemble

In [21]:
df_orders["Customer Name"] = df_orders["Customer Name"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))
df_orders["Product Name"] = df_orders["Product Name"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

In [22]:
print(f"Qtd. Valores Únicos na coluna Customer Name : {len(df_orders["Customer Name"].unique())}")
print(f"Qtd. Valores Únicos na coluna Product Name : {len(df_orders["Product Name"].unique())}")

Qtd. Valores Únicos na coluna Customer Name : 795
Qtd. Valores Únicos na coluna Product Name : 3784


##### Colunas com datas

In [23]:
print(f"Order Date : {df_orders["Order Date"].dtype}")
print(f"Ship Date : {df_orders["Ship Date"].dtype}")

Order Date : datetime64[ns]
Ship Date : datetime64[ns]


In [24]:
df_orders["Order Date"] = pd.to_datetime(df_orders["Order Date"]).dt.date
df_orders["Ship Date"] = pd.to_datetime(df_orders["Ship Date"]).dt.date

In [25]:
print(f"Order Date : {df_orders["Order Date"].dtype}")
print(f"Ship Date : {df_orders["Ship Date"].dtype}")

Order Date : object
Ship Date : object


##### Colunas Categóricas

In [26]:
print(f"Qtd. Valores Únicos:\n")
print(f"\tShip Mode : {len(df_orders["Ship Mode"].unique())}")
print(f"\tSegment : {len(df_orders["Segment"].unique())}")
print(f"\tMarket : {len(df_orders["Market"].unique())}")
print(f"\tCategory : {len(df_orders["Category"].unique())}")
print(f"\tSub-Category : {len(df_orders["Sub-Category"].unique())}")
print(f"\tOrder Priority : {len(df_orders["Order Priority"].unique())}")
print(f"\tCity : {len(df_orders["City"].unique())}")
print(f"\tState : {len(df_orders["State"].unique())}")
print(f"\tCountry : {len(df_orders["Country"].unique())}")
print(f"\tRegion : {len(df_orders["Region"].unique())}")

Qtd. Valores Únicos:

	Ship Mode : 4
	Segment : 3
	Market : 7
	Category : 3
	Sub-Category : 17
	Order Priority : 4
	City : 3636
	State : 1094
	Country : 147
	Region : 13


In [27]:
# Analisando valores da coluna Ship Mode
for categoria in df_orders["Ship Mode"].unique():
    print(categoria)

Same Day
Second Class
First Class
Standard Class


In [28]:
# Analisando valores da coluna Segment
for categoria in df_orders["Segment"].unique():
    print(categoria)

Consumer
Corporate
Home Office


In [29]:
# Analisando valores da coluna Market
for categoria in df_orders["Market"].unique():
    print(categoria)

US
APAC
EU
Africa
EMEA
LATAM
Canada


In [30]:
# Analisando valores da coluna Category
for categoria in df_orders["Category"].unique():
    print(categoria)

Technology
Furniture
Office Supplies


In [31]:
# Analisando valores da coluna Sub-Category
for categoria in df_orders["Sub-Category"].unique():
    print(categoria)

Accessories
Chairs
Phones
Copiers
Tables
Binders
Supplies
Appliances
Machines
Bookcases
Storage
Furnishings
Art
Paper
Envelopes
Fasteners
Labels


In [32]:
# Analisando valores da coluna Order Priority
for categoria in df_orders["Order Priority"].unique():
    print(categoria)

Critical
Medium
High
Low


In [33]:
# Analisando valores da coluna City
for categoria in df_orders["City"].unique():
    print(categoria)

New York City
Wollongong
Brisbane
Berlin
Dakar
Sydney
Porirua
Hamilton
Sacramento
Concord
Alexandria
Kabul
Jizan
Toledo
Mudanjiang
Paris
Henderson
Prato
Townsville
Uvinza
Bytom
Chicago
Suzhou
Edinburgh
Juárez
Soyapango
Taipei
Leipzig
Los Angeles
Surat
Santo Domingo
Saint-Brieuc
Amarillo
Gold Coast
Fresno
Kamina
Burlington
Stockton-on-Tees
Mataram
Gorakhpur
Thiruvananthapuram
Huddersfield
Minneapolis
Montreuil
Shouguang
Jamshedpur
Paysandú
Behshahr
Huntington Beach
Maputo
Bhopal
Seattle
Delhi
Geraldton
Tallahassee
Dhaka
Munster
Celle
Wuxi
Richmond
Seville
Raipur
Gómez Palacio
Kharkiv
Jinan
Chinandega
Kananga
Palembang
London
Melbourne
Atlanta
Duisburg
Nanchong
Naihati
Lille
Meknes
Jackson
Philadelphia
Krefeld
Bandung
Casablanca
Tongi
Montréal
Manila
Newcastle
Graz
Nowra
Boulogne-Billancourt
Malakoff
Kinshasa
Perth
Le Bouscat
Puebla
Augsburg
Nice
Medellín
Bergen op Zoom
Lakewood
Hanover
Vigo
Gaoyou
Bremen
Muret
Zigong
Adelaide
Detroit
Chelles
Kuantan
Harrisonburg
Everett
Quito
Vadodara
S

In [34]:
df_orders["City"] = df_orders["City"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

In [35]:
# Analisando valores da coluna State
for categoria in df_orders["State"].unique():
    print(categoria)

New York
New South Wales
Queensland
Berlin
Dakar
Wellington
Waikato
California
North Carolina
Virginia
Kabul
Jizan
Parana
Heilongjiang
Ile-de-France
Kentucky
Tuscany
Kigoma
Silesia
Illinois
Anhui
Scotland
Chihuahua
San Salvador
Taipei City
Saxony
Gujarat
Santo Domingo
Brittany
Texas
Katanga
England
Nusa Tenggara Barat
Haryana
Kerala
Minnesota
Shandong
Jharkhand
Paysandú
Mazandaran
Cidade De Maputo
Madhya Pradesh
Washington
Delhi
Western Australia
Florida
Dhaka
Lower Saxony
Jiangsu
Andalusía
Uttarakhand
Durango
Kharkiv
Chinandega
Kasai-Occidental
Sumatera Selatan
Victoria
Georgia
North Rhine-Westphalia
Sichuan
West Bengal
Nord-Pas-de-Calais
Meknès-Tafilalet
Michigan
Pennsylvania
Jawa Barat
Grand Casablanca
Nevada
Quebec
National Capital
Styria
Kinshasa
Aquitaine
Puebla
Bavaria
Provence-Alpes-Côte d'Azur
Antioquia
North Brabant
New Jersey
Galicia
Bremen
Midi-Pyrénées
South Australia
Pahang
Massachusetts
Pichincha
Hainan
Bangkok
Sonora
Hubei
Wisconsin
São Paulo
Veneto
Woqooyi Galbeed
Yogy

In [36]:
df_orders["State"] = df_orders["State"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

In [37]:
# Analisando valores da coluna Country
for categoria in df_orders["Country"].unique():
    print(categoria)

United States
Australia
Germany
Senegal
New Zealand
Afghanistan
Saudi Arabia
Brazil
China
France
Italy
Tanzania
Poland
United Kingdom
Mexico
El Salvador
Taiwan
India
Dominican Republic
Democratic Republic of the Congo
Indonesia
Uruguay
Iran
Mozambique
Bangladesh
Spain
Ukraine
Nicaragua
Morocco
Canada
Philippines
Austria
Colombia
Netherlands
Malaysia
Ecuador
Thailand
Somalia
Guatemala
Belarus
Cambodia
South Africa
Japan
Russia
Egypt
Azerbaijan
Lithuania
Argentina
Lesotho
Vietnam
Cuba
Romania
Turkey
Cameroon
Hungary
Singapore
Angola
Belgium
Pakistan
Finland
Ghana
Zambia
Iraq
Liberia
Georgia
Switzerland
Albania
Chad
Montenegro
Namibia
Portugal
Madagascar
Sweden
Myanmar (Burma)
Jamaica
Qatar
Republic of the Congo
Norway
Algeria
South Korea
Nigeria
Estonia
Cote d'Ivoire
Honduras
Paraguay
Czech Republic
Central African Republic
Benin
Bolivia
Chile
Martinique
Syria
Lebanon
Kenya
Mali
Libya
Venezuela
Trinidad and Tobago
Ireland
Bulgaria
Panama
Israel
Haiti
Barbados
Slovenia
Togo
Mauritania
Gui

In [38]:
df_orders["Country"] = df_orders["Country"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

In [39]:
# Analisando valores da coluna Region
for categoria in df_orders["Region"].unique():
    print(categoria)

East
Oceania
Central
Africa
West
South
Central Asia
EMEA
North Asia
North
Caribbean
Southeast Asia
Canada


In [40]:
df_orders["Region"] = df_orders["Region"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

In [41]:
print(f"Qtd. Valores Únicos:\n")
print(f"\tShip Mode : {len(df_orders["Ship Mode"].unique())}")
print(f"\tSegment : {len(df_orders["Segment"].unique())}")
print(f"\tMarket : {len(df_orders["Market"].unique())}")
print(f"\tCategory : {len(df_orders["Category"].unique())}")
print(f"\tSub-Category : {len(df_orders["Sub-Category"].unique())}")
print(f"\tOrder Priority : {len(df_orders["Order Priority"].unique())}")
print(f"\tCity : {len(df_orders["City"].unique())}")
print(f"\tState : {len(df_orders["State"].unique())}")
print(f"\tCountry : {len(df_orders["Country"].unique())}")
print(f"\tRegion : {len(df_orders["Region"].unique())}")

Qtd. Valores Únicos:

	Ship Mode : 4
	Segment : 3
	Market : 7
	Category : 3
	Sub-Category : 17
	Order Priority : 4
	City : 3633
	State : 1093
	Country : 147
	Region : 13


In [42]:
df_orders["Ship Mode"] = df_orders["Ship Mode"].astype("category")
print(f"Ship Mode : {df_orders["Ship Mode"].dtype}")

df_orders["Segment"] = df_orders["Segment"].astype("category")
print(f"Segment : {df_orders["Segment"].dtype}")

df_orders["Market"] = df_orders["Market"].astype("category")
print(f"Market : {df_orders["Market"].dtype}")

df_orders["Category"] = df_orders["Category"].astype("category")
print(f"Category : {df_orders["Category"].dtype}")

df_orders["Sub-Category"] = df_orders["Sub-Category"].astype("category")
print(f"Sub-Category : {df_orders["Sub-Category"].dtype}")

df_orders["Order Priority"] = df_orders["Order Priority"].astype("category")
print(f"Order Priority : {df_orders["Order Priority"].dtype}")

df_orders["City"] = df_orders["City"].astype("category")
print(f"City : {df_orders["City"].dtype}")

df_orders["State"] = df_orders["State"].astype("category")
print(f"State : {df_orders["State"].dtype}")

df_orders["Country"] = df_orders["Country"].astype("category")
print(f"Country : {df_orders["Country"].dtype}")

df_orders["Region"] = df_orders["Region"].astype("category")
print(f"Region : {df_orders["Region"].dtype}")

Ship Mode : category
Segment : category
Market : category
Category : category
Sub-Category : category
Order Priority : category
City : category
State : category
Country : category
Region : category


##### Colunas Numéricas

In [43]:
df_orders["Discount"].unique()

array([0.   , 0.1  , 0.2  , 0.4  , 0.15 , 0.3  , 0.5  , 0.17 , 0.47 ,
       0.25 , 0.002, 0.07 , 0.32 , 0.27 , 0.7  , 0.35 , 0.15 , 0.6  ,
       0.65 , 0.8  , 0.57 , 0.37 , 0.402, 0.55 , 0.202, 0.45 , 0.45 ,
       0.602, 0.85 ])

In [44]:
df_orders["Discount"] = df_orders["Discount"].astype(str).apply(lambda x: re.sub(r'[^\d.]', '', x.replace(",", ".")))
df_orders["Discount"] = df_orders["Discount"].apply(pd.to_numeric, downcast="float")
df_orders["Discount"].unique()

array([0.   , 0.1  , 0.2  , 0.4  , 0.15 , 0.3  , 0.5  , 0.17 , 0.47 ,
       0.25 , 0.002, 0.07 , 0.32 , 0.27 , 0.7  , 0.35 , 0.6  , 0.65 ,
       0.8  , 0.57 , 0.37 , 0.402, 0.55 , 0.202, 0.45 , 0.602, 0.85 ],
      dtype=float32)

In [45]:
df_orders["Sales"].unique()

array([2.309650e+03, 3.709395e+03, 5.175171e+03, ..., 1.624000e+00,
       5.364000e+00, 4.440000e-01])

In [46]:
df_orders["Sales"] = df_orders["Sales"].astype(str).apply(lambda x: re.sub(r'[^\d.]', '', x.replace(",", ".")))
df_orders["Sales"] = df_orders["Sales"].apply(pd.to_numeric, downcast="float")
df_orders["Sales"].unique()

array([2.30964990e+03, 3.70939502e+03, 5.17517090e+03, ...,
       2.04160004e+01, 1.62399995e+00, 4.44000006e-01])

In [47]:
df_orders["Sales"].dtypes

dtype('float64')

In [48]:
df_orders["Shipping Cost"].unique()

array([9.3357e+02, 9.2363e+02, 9.1549e+02, ..., 1.0000e-02, 3.0000e-03,
       2.0000e-03])

In [49]:
df_orders["Shipping Cost"] = df_orders["Shipping Cost"].astype(str).apply(lambda x: re.sub(r'[^\d.]', '', x.replace(",", ".")))
df_orders["Shipping Cost"] = df_orders["Shipping Cost"].apply(pd.to_numeric, downcast="float")
df_orders["Shipping Cost"].unique()

array([9.3357e+02, 9.2363e+02, 9.1549e+02, ..., 1.0000e-02, 3.0000e-03,
       2.0000e-03], dtype=float32)

In [50]:
df_orders["Profit"].unique()

array([ 762.1845, -288.765 ,  919.971 , ...,   -4.466 ,   -6.456 ,
        -49.572 ])

In [51]:
df_orders["Profit"] = df_orders["Profit"].astype(str).apply(lambda x: re.sub(r'[^\d.]', '', x.replace(",", ".")))
df_orders["Profit"] = df_orders["Profit"].apply(pd.to_numeric, downcast="float")
df_orders["Profit"].unique()

array([7.621845e+02, 2.887650e+02, 9.199710e+02, ..., 4.704000e-01,
       4.466000e+00, 4.957200e+01], dtype=float32)

In [52]:
df_orders["Quantity"].unique()

array([ 7,  9,  5,  8,  4,  6, 13, 12, 14, 10,  2, 11,  3,  1])

In [53]:
df_orders["Quantity"].dtypes

dtype('int64')

In [54]:
df_orders["Quantity"] = df_orders["Quantity"].astype(str).apply(lambda x: re.sub(r'[^\d.]', '', x.replace(",", "."))).astype(float).round(0).astype(int)
df_orders["Quantity"] = df_orders["Quantity"].apply(pd.to_numeric, downcast="integer")
df_orders["Quantity"].unique()

array([ 7,  9,  5,  8,  4,  6, 13, 12, 14, 10,  2, 11,  3,  1])

In [55]:
df_orders["Quantity"].dtypes

dtype('int64')

#### Removendo registros duplicados

In [56]:
df_orders.shape

(51290, 22)

In [57]:
df_orders = df_orders.drop_duplicates()

In [58]:
df_orders.shape

(51290, 22)

#### Identificando e exportando candidatos a outliers

In [59]:
dict_cand_outliers_orders = {}
for coluna in df_orders.select_dtypes(include="number").columns:

    media = df_orders.describe().loc['50%', coluna]
    desvio_padrao = df_orders.describe().loc['std', coluna]

    lista_cand_outliers_orders = [x for x in df_orders[coluna] if (x < media - 3 * desvio_padrao) or (x > media + 3 * desvio_padrao)]

    dict_cand_outliers_orders[coluna] = lista_cand_outliers_orders

maior_qtd_cand_outliers_orders = 0
for coluna in dict_cand_outliers_orders.keys():

    if len(dict_cand_outliers_orders[coluna]) > maior_qtd_cand_outliers_orders:

        maior_qtd_cand_outliers_orders = len(dict_cand_outliers_orders[coluna])

for coluna in dict_cand_outliers_orders.keys():

    while len(dict_cand_outliers_orders[coluna]) < maior_qtd_cand_outliers_orders:

        dict_cand_outliers_orders[coluna].append(np.nan)

df_cand_outliers_orders = pd.DataFrame(dict_cand_outliers_orders)
df_cand_outliers_orders

,Sales,Quantity,Discount,Profit,Shipping Cost
0,2309.649902,13.0,0.70,762.184509,933.570007
1,3709.395020,12.0,0.70,919.971008,923.630005
2,5175.170898,14.0,0.65,763.275024,915.489990
3,2892.510010,10.0,0.70,564.840027,910.159973
4,2832.959961,12.0,0.80,996.479980,903.039978
...,...,...,...,...,...
2116,NaN,NaN,0.80,NaN,NaN
2117,NaN,NaN,0.80,NaN,NaN
2118,NaN,NaN,0.70,NaN,NaN
2119,NaN,NaN,0.70,NaN,NaN


In [60]:
df_cand_outliers_orders.to_parquet(r'..\data\df_cand_outliers_orders.parquet')

#### Exportando df_orders tratado

In [61]:
df_orders.to_parquet(r'..\data\df_orders.parquet')

### Tratando conjunto de dados Returns

#### Analisando conjunto de dados

In [62]:
df_returns = planilhas["Returns"]
df_returns.head(3)

,Returned,Order ID,Market
0,Yes,MX-2013-168137,LATAM
1,Yes,US-2011-165316,LATAM
2,Yes,ES-2013-1525878,EU


In [63]:
df_returns.describe()

,Returned,Order ID,Market
count,1173,1173,1173
unique,1,1172,4
top,Yes,US-2014-136679,LATAM
freq,1173,2,297


In [64]:
df_returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Returned  1173 non-null   object
 1   Order ID  1173 non-null   object
 2   Market    1173 non-null   object
dtypes: object(3)
memory usage: 27.6+ KB


#### Tratando tipos de dados e possíveis inconsistências de registro

##### Coluna Market

In [65]:
for categoria in df_returns["Market"].unique():

    if categoria not in df_orders["Market"].unique():

        match categoria:

            case "United States":

                df_returns["Market"] = df_returns["Market"].replace("United States", "US")
            
            case _ :

                print(f"CATEGORIA DESCONHECIDA : {categoria}")

In [66]:
df_returns["Market"] = df_returns["Market"].astype("category")
df_returns["Market"].unique()

['LATAM', 'EU', 'US', 'APAC']
Categories (4, object): ['APAC', 'EU', 'LATAM', 'US']

##### Coluna Order ID

In [67]:
for order_id in df_returns["Order ID"].unique():

    if order_id not in df_orders["Order ID"].unique():

        print(f"ORDER ID DESCONHECIDO : {order_id}")

In [68]:
df_returns["Order ID"] = df_returns["Order ID"].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z0-9-]', '', unidecode(x.upper())))
df_returns["Order ID"].unique()

array(['MX-2013-168137', 'US-2011-165316', 'ES-2013-1525878', ...,
       'ES-2012-3246286', 'ES-2012-4379168', 'CA-2014-168193'],
      dtype=object)

##### Coluna Returned

In [69]:
df_returns["Returned"].unique()

array(['Yes'], dtype=object)

In [70]:
df_returns["Returned"] = "Devolvido"
df_returns["Returned"] = df_returns["Returned"].astype("category")
df_returns["Returned"]

0       Devolvido
1       Devolvido
2       Devolvido
3       Devolvido
4       Devolvido
          ...    
1168    Devolvido
1169    Devolvido
1170    Devolvido
1171    Devolvido
1172    Devolvido
Name: Returned, Length: 1173, dtype: category
Categories (1, object): ['Devolvido']

#### Removendo registros duplicados

In [71]:
df_returns = df_returns.drop_duplicates()

#### Exportando df_returns tratado

In [72]:
df_returns.to_parquet(r'..\data\df_returns.parquet')

### Tratando conjunto de dados People

#### Analisando conjunto de dados

In [73]:
df_people = planilhas["People"]
df_people

,Person,Region
0,Anna Andreadi,Central
1,Chuck Magee,South
2,Kelly Williams,East
3,Matt Collister,West
4,Deborah Brumfield,Africa
5,Larry Hughes,AMEA
6,Nicole Hansen,Canada
7,Giulietta Dortch,Caribbean
8,Nora Preis,Central Asia
9,Jack Lebron,North


In [74]:
df_people.describe()

,Person,Region
count,13,13
unique,13,13
top,Anna Andreadi,Central
freq,1,1


In [75]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Person  13 non-null     object
 1   Region  13 non-null     object
dtypes: object(2)
memory usage: 340.0+ bytes


#### Tratando tipos de dados e possíveis inconsistências de registro

##### Coluna Person

In [76]:
df_people["Person"] = df_people["Person"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

##### Coluna Region

In [77]:

df_people["Region"] = df_people["Region"].astype(str).apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zA-Z0-9\s]', '', unidecode(x.strip().title()))))

In [78]:
for regiao in df_people["Region"].unique():

    if regiao not in df_orders["Region"].unique():

        print(f"REGIAO DESCONHECIDA : {regiao}")

REGIAO DESCONHECIDA : Amea


In [79]:
for r in df_orders["Region"].unique():
    print(r)

East
Oceania
Central
Africa
West
South
Central Asia
Emea
North Asia
North
Caribbean
Southeast Asia
Canada


In [80]:
df_people.loc[df_people["Region"] == "Amea", "Region"] = "Emea"
df_people["Region"] = df_people["Region"].astype("category")

#### Exportando df_people tratado

In [81]:
df_people.to_parquet(r'..\data\df_people.parquet')

### Unificando conjuntos de dados

#### Analisando conjunto de dados

In [82]:
df_completo = df_orders.merge(df_returns, 
                              on=["Order ID", "Market"], 
                              how="left")
df_completo.head(3)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Returned
0,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,...,Technology,Accessories,Plantronics Cs510 OverTheHead Monaural Wireles...,2309.649902,7,0.0,762.184509,933.570007,Critical,NaN
1,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,...,Furniture,Chairs,Novimex Executive Leather Armchair Black,3709.395020,9,0.1,288.765015,923.630005,Critical,Devolvido
2,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,...,Technology,Phones,Nokia Smart Phone With Caller Id,5175.170898,9,0.1,919.971008,915.489990,Medium,NaN


In [83]:
df_completo = df_completo.merge(df_people, 
                              on="Region",	 
                              how="left")
df_completo.head(3)

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,...,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Returned,Person
0,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,...,Accessories,Plantronics Cs510 OverTheHead Monaural Wireles...,2309.649902,7,0.0,762.184509,933.570007,Critical,NaN,Kelly Williams
1,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,...,Chairs,Novimex Executive Leather Armchair Black,3709.395020,9,0.1,288.765015,923.630005,Critical,Devolvido,Anthony Jacobs
2,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,...,Phones,Nokia Smart Phone With Caller Id,5175.170898,9,0.1,919.971008,915.489990,Medium,NaN,Anthony Jacobs


#### Tratando valores ausentes

In [84]:
df_completo.isna().sum()

Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Market                0
Region                0
Product ID            0
Category              0
Sub-Category          0
Product Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
Returned          48247
Person                0
dtype: int64

In [85]:
df_completo["Returned"] = df_completo["Returned"].astype(str)
df_completo.loc[df_completo["Returned"].isna(),"Returned"] = "Não Devolvido"

#### Tratando tipos de dados

In [86]:
df_completo["Order ID"] = df_completo["Order ID"].astype(str)
df_completo["Customer ID"] = df_completo["Customer ID"].astype(str)
df_completo["Product ID"] = df_completo["Product ID"].astype(str)
df_completo["Customer Name"] = df_completo["Customer Name"].astype(str)
df_completo["Product Name"] = df_completo["Product Name"].astype(str)
df_completo["Person"] = df_completo["Person"].astype(str)
df_completo["Order Date"] = pd.to_datetime(df_completo["Order Date"]).dt.date
df_completo["Ship Date"] = pd.to_datetime(df_completo["Ship Date"]).dt.date
df_completo["Ship Mode"] = df_completo["Ship Mode"].astype("category")
df_completo["Segment"] = df_completo["Segment"].astype("category")
df_completo["Market"] = df_completo["Market"].astype("category")
df_completo["Category"] = df_completo["Category"].astype("category")
df_completo["Sub-Category"] = df_completo["Sub-Category"].astype("category")
df_completo["Order Priority"] = df_completo["Order Priority"].astype("category")
df_completo["City"] = df_completo["City"].astype("category")
df_completo["State"] = df_completo["State"].astype("category")
df_completo["Country"] = df_completo["Country"].astype("category")
df_completo["Region"] = df_completo["Region"].astype("category")
df_completo["Quantity"] = df_completo["Quantity"].apply(pd.to_numeric, downcast="integer")
df_completo["Profit"] = df_completo["Profit"].apply(pd.to_numeric, downcast="float")
df_completo["Shipping Cost"] = df_completo["Shipping Cost"].apply(pd.to_numeric, downcast="float")
df_completo["Sales"] = df_completo["Sales"].apply(pd.to_numeric, downcast="float")
df_completo["Discount"] = df_completo["Discount"].apply(pd.to_numeric, downcast="float")
df_completo["Returned"] = df_completo["Returned"].astype("category")

#### Exportando df_completo tratado

In [87]:
df_completo.to_parquet(r'..\data\df_completo.parquet')